In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.append(os.path.join(os.path.abspath(''), '../'))

from models.graphsage_model import GraphSAGE
from pprint import pprint
from dataset.data_loader import DataLoader, playtime_forever_users_games_edge_scoring_function, GaussianNormalizer

from utils.utils import get_game_name_and_scores

/Users/jackson/.pyenv/versions/3.11.3/envs/ml_project_3.11.3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%reload_ext autoreload

In [3]:
data_loader = DataLoader(users_games_edge_scoring_function=playtime_forever_users_games_edge_scoring_function,user_game_edge_embeddings=['playtime_forever'], users_games_edge_score_normalizers = [GaussianNormalizer(1.0, 1.0)], interactions_score_normalizers = [GaussianNormalizer(0.0, 1.0)], cache_local_dataset=True, game_embeddings=['name'])
# data_loader = DataLoader(users_games_edge_scoring_function=playtime_forever_users_games_edge_scoring_function, user_game_edge_embeddings=['playtime_forever'], users_games_edge_score_normalizers = [GaussianNormalizer(1.0, 1.0)], interactions_score_normalizers = [GaussianNormalizer(0.0, 1.0)], cache_local_dataset=True, game_embeddings=['numReviews', 'avgReviewScore', 'price', 'numFollowers', 'genres', 'tags', 'name'])
data_loader.load_random_edge_train_test_split(train_percentage=0.9, test_percentage=0.1, seed=0)
data_loader.save_data_loader_parameters('test_graphsage_data_loader', overwrite=True)

In [4]:
%reload_ext autoreload
save_file_name = 'test_graphsage_model_TEMP'
model = GraphSAGE(hidden_channels=50, game_embedding_size=50, aggr='mean', save_file_name=save_file_name, nn_save_name='best', num_epochs=25, batch_percent=0.1, learning_rate=1e-3, weight_decay=1e-10, seed=12412)
model.set_data_loader(data_loader)
model.train(debug=False)
model.save(save_file_name, overwrite=True)

Total Learnable Parameters: 1729800


Training: 100%|██████████| 25/25 [26:15<00:00, 63.02s/it]


In [13]:
# test_user = 76561199048589805 # Only 1 game
# test_user = 76561198835352289 # Only 4 games
# test_user = 76561198880605436 # ~20 games
test_user = 76561199040442492 # ~400 games
display(get_game_name_and_scores(data_loader, model.score_and_predict_n_games_for_user(test_user)[:10]))
display(get_game_name_and_scores(data_loader, model.score_and_predict_n_games_for_user(test_user)[-10:]))

,id,name,score
0,4000,Garry's Mod,3.185205e-21
1,41700,S.T.A.L.K.E.R.: Call of Pripyat,3.185205e-21
2,92800,SpaceChem,3.185205e-21
3,400,Portal,3.185205e-21
4,620,Portal 2,3.185205e-21
5,105600,Terraria,3.185205e-21
6,24240,PAYDAY™ The Heist,3.185205e-21
7,48000,LIMBO,3.185205e-21
8,206440,To the Moon,3.185205e-21
9,4500,S.T.A.L.K.E.R.: Shadow of Chernobyl,3.185205e-21


,id,name,score
0,2635820,Krosfighter,0.0
1,2693730,Dawnsbury Days,0.0
2,2735060,Station 5,0.0
3,2753820,My step family,0.0
4,2767520,LurkBait Twitch Fishing,0.0
5,2789810,Bingle Bingle,0.0
6,2834180,Sex Faculty🔞,0.0
7,1785650,TopSpin 2K25,0.0
8,2496590,Untraveled Lands: Chantico,0.0
9,1912750,Terra Memoria,0.0


In [6]:
model2 = GraphSAGE()
model2.load('test_graphsage_model_TEMP', load_published_model=False)
model2.set_data_loader(data_loader)

In [7]:
# test_user = 76561199048589805 # Only 1 game
# test_user = 76561198835352289 # Only 4 games
# test_user = 76561198880605436 # ~20 games
test_user = 76561199040442492 # ~400 games
display(get_game_name_and_scores(data_loader, model2.score_and_predict_n_games_for_user(test_user)[:10]))
display(get_game_name_and_scores(data_loader, model2.score_and_predict_n_games_for_user(test_user)[-10:]))

,id,name,score
0,615010,Runic Rampage - Action RPG,1.076071
1,1614940,Black Smith3,1.074266
2,1273690,Ashwalkers,1.072257
3,2328480,Ascension,1.066662
4,1674920,Sovereign Syndicate,1.066263
5,283580,New York Bus Simulator,1.063507
6,376490,The Great Gatsby: Secret Treasure,1.063019
7,2379780,Balatro,1.062290
8,544650,Sacred Almanac Traces of Greed,1.062023
9,40320,Rush for Berlin Gold,1.061551


,id,name,score
0,911400,Assassin's Creed® III Remastered,1.844701e-04
1,582010,Monster Hunter: World,1.491923e-04
2,367520,Hollow Knight,1.348267e-04
3,1446780,MONSTER HUNTER RISE,6.039882e-05
4,377160,Fallout 4,3.419242e-05
5,359550,Tom Clancy's Rainbow Six® Siege,2.339051e-05
6,570,Dota 2,1.239838e-05
7,292030,The Witcher® 3: Wild Hunt,5.512095e-06
8,4000,Garry's Mod,8.770490e-07
9,1599340,Lost Ark,1.016726e-11


In [8]:
user_to_fine_tune = 76561198166465514 # 76561198103368250
fine_tune_model = GraphSAGE()
fine_tune_model.load('test_graphsage_model_TEMP', load_published_model=False)
fine_tune_data_loader = DataLoader.load_from_file('test_graphsage_data_loader', use_published_models_path=False, load_live_data_loader=True)
fine_tune_model.set_data_loader(fine_tune_data_loader)
fine_tune_model.fine_tune(user_to_fine_tune, debug=False)
display(get_game_name_and_scores(fine_tune_data_loader, fine_tune_model.score_and_predict_n_games_for_user(user_to_fine_tune)[:10]))
display(get_game_name_and_scores(fine_tune_data_loader, fine_tune_model.score_and_predict_n_games_for_user(user_to_fine_tune)[-10:]))

,id,name,score
0,352850,DGU: Death God University,16.380098
1,1294660,Divine Knockout (DKO),14.477905
2,615010,Runic Rampage - Action RPG,12.561198
3,1831630,建筑吧公司,12.554072
4,1614940,Black Smith3,12.529692
5,509800,The Last Weekend,12.481660
6,961200,Predecessor,12.474713
7,739660,Battle Trendaria,12.453715
8,345860,Super Galaxy Squadron EX Turbo,12.452411
9,747590,Drive//Shaft,12.435960


,id,name,score
0,212070,Star Conflict,4.007316e-07
1,302670,Call to Arms,3.872192e-07
2,393530,Insanity Clicker,2.699618e-07
3,253980,Enclave,2.270936e-07
4,534380,Dying Light 2 Stay Human: Reloaded Edition,1.372062e-07
5,205910,Tiny and Big: Grandpa's Leftovers,1.284405e-07
6,1105670,The Last Spell,1.283282e-07
7,1366540,Dyson Sphere Program,1.283134e-07
8,1267910,Melvor Idle,4.716061e-08
9,570940,DARK SOULS™: REMASTERED,2.674644e-08
